In [26]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt

In [35]:
select = 'valid'
iou_threshold = 0.5

GT_JSON = f'../../../Split_data/{select}_0_5.json'
PRED_CSV = './result/paste/deta-swin-large_3_img_size_720_test_hFlip.csv' #./result/TTA/valid/deta-swin-large_0_img_size_720_hflip.csv'

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal",
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]




In [40]:
pred[0]

['test/0000.jpg', '7', '0.7866849', 603.3789, 955.8824, 517.5762, 1023.17944]

In [38]:
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

pred = []
file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).strip().split(' '))

    if len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    elif isinstance(bbox, float):
        print(f'{file_name} empty box')
        continue
    else:
        raise Exception('error', 'invalid box type')
    
    for box in boxes:
        pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])


1942it [00:00, 5092.53it/s]

test/1474.jpg empty box


4871it [00:01, 3952.86it/s]


NameError: name 'prds' is not defined

In [ ]:
gt = []
coco = COCO(GT_JSON)

for image_id in coco.getImgIds():
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
    file_name = image_info['file_name']

    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                float(annotation['bbox'][0]),
                float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                float(annotation['bbox'][1]),
                float(annotation['bbox'][1]) + float(annotation['bbox'][3])])
        
mean_ap, average_precisions = mean_average_precision_for_boxes(gt, pred, iou_threshold=iou_threshold)
print(f'mAP@{iou_threshold*100:.0f}: {mean_ap}')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
